In [17]:
from dotenv import load_dotenv
from typing import TypedDict, Literal, Annotated
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from IPython.display import display, Image
from langgraph.graph.message import add_messages


load_dotenv()

True

In [18]:


llm = init_chat_model(model="llama3.1:8b", model_provider="ollama")

class State(TypedDict):
     messages: Annotated[list, add_messages]


def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}



builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)
graph = builder.compile()

In [19]:
message = {"role": "user", "content": "First man landed on the moon?"}
state = {"messages": [message]}
response = graph.invoke(state)

response["messages"]

[HumanMessage(content='First man landed on the moon?', additional_kwargs={}, response_metadata={}, id='33bc7eda-68f5-4b05-b047-f80f91fc4a05'),
 AIMessage(content='The first man to land on the moon was Neil Armstrong, an American astronaut who commanded the Apollo 11 mission. On July 20, 1969, Armstrong stepped out of the lunar module Eagle and onto the moon\'s surface, famously declaring:\n\n"That\'s one small step for man, one giant leap for mankind."\n\nArmstrong was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the moon during the mission. Michael Collins remained in orbit around the moon in the command module Columbia.\n\nThe Apollo 11 mission marked a historic achievement in space exploration and paved the way for future manned missions to the moon and beyond!', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-30T06:46:00.1376597Z', 'done': True, 'done_reason': 'stop', 'total_duration': 26136282700, 'load_duration': 897

In [ ]:
state = None
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    if state is None:
        state = {"messages": [{"role": "user", "content": user_input}]}
    else:
        state["messages"].append({"role": "user", "content": user_input})
        
    response = graph.invoke(state)
    for msg in response["messages"]:
        print(f"{msg['role']}: {msg['content']}")